In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("qbbo.ipynb")

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import otter
grader = otter.Notebook()

QBBO_PATH = '04222019.QBBO_21_C-decal'

# Parsing QBBO

QBBO is NASDAQ's level 1 market data protocol. You can [read its specification online](https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/QBBOSpecification2.1.pdf). This protocol consists of a repeated series of a message length then a message.

Let's start with some helper functions:

In [3]:
import datetime

def read_int(raw_bytes):
    return int.from_bytes(raw_bytes, byteorder='big')

def read_string(raw_bytes):
    return raw_bytes.decode('ascii')

def parse_timestamp(ns_since_midnight):
    midnight = datetime.datetime(2019, 4, 22)
    delta = datetime.timedelta(seconds=ns_since_midnight / 1e9)
    return midnight + delta

**Question 1:** QBBO consists of a series of messages, each consisting of two bytes for length. For example, see the following:

```
00 06 aa bb cc dd ee ff 00 04 14 01 48 03
[len] [    message    ] [len] [ message ]
```

The above contains two messages:
1. `aa bb cc dd ee ff`
2. `14 01 48 03`

A file `04222019.QBBO_21_C-decal` has been provided alongside this assignemnt. It contains real NASDAQ QBBO market data, filtered to only contain system messages and a few tickers (to reduce the size of the download).

Read this file, yielding each message's content.

In [4]:
def read_qbbo_messages(file_path):
    with open(file_path, 'rb') as file:
        while True:
            len_bytes = file.read(2)
            
            message_length = read_int(len_bytes)
            message = file.read(message_length)

            yield message

In [5]:
test_reader = read_qbbo_messages(QBBO_PATH)
assert next(test_reader) == b'S\x00\x00\n\x04L`09O'
assert next(test_reader) == b'S\x00\x00\r\x18\xc2\xe5B\xedS'

In [6]:
grader.check("q1")

q1 results: All test cases passed!

**Question 2:** Parse system event messages according to the spec.

In [7]:
def parse_system_event(message):
    event_code_map = {
        'O': 'Start of Transmissions', 
        'S': 'Start of System Hours', 
        'Q': 'Start of Market Hours',
        'M': 'End of Market Hours',
        'E': 'End of System Hours',
        'C': 'End of Transmissions',
    }
    
    message_type = message[0:1]
    tracking_no = message[1:3]
    timestamp = message[3:9]
    event_code = message[9:10]

    return {
        'type': 'S',
        'timestamp':parse_timestamp(read_int(timestamp)),
        'event': event_code_map[read_string(event_code)]
    }

In [8]:
grader.check("q2")

q2 results: All test cases passed!

**Question 3:** Parse quotation messages

In [9]:
def parse_quotation(message):
    timestamp = parse_timestamp(read_int(message[3:9]))
    ticker = read_string(message[9:17]).strip()
    bid_price = read_int(message[18:22]) / 10000
    bid_size = read_int(message[22:26])
    ask_price = read_int(message[26:30]) / 10000
    ask_size = read_int(message[30:34])
    
    return {
        'type': 'Q',
        'timestamp': timestamp,
        'ticker': ticker,
        'bid_price': bid_price,
        'bid_size': bid_size,
        'ask_price': ask_price,
        'ask_size': ask_size
    }


In [10]:
assert parse_quotation(b'Q\x00\x00\r\x18\xc3\xe4\xa9GMSFT    Q\x00\x0c5\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00\x00') == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [11]:
grader.check("q3")

q3 results: All test cases passed!

**Question 4:** Parse a message using its message type (the first character). Use the functions you wrote above.

In [12]:
def parse_message(message):
    event_type = read_string(message[:1])
    if event_type == 'S':
        return parse_system_event(message)
    elif event_type == 'Q':
        return parse_quotation(message)
    else:
        return {
            'type': 'Others',
            'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
        }
        
    

In [13]:
assert parse_message(b'S\x00\x00\n\x04L`09O') == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 3, 3, 33, 577519),
    'event': 'Start of Transmissions',
}

In [14]:
assert parse_message(b'Q\x00\x00\r\x18\xc3\xe4\xa9GMSFT    Q\x00\x0c5\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00\x00') == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [15]:
grader.check("q4")

q4 results: All test cases passed!

**Question 5:** Now, write a function to read a QBBO file and yield parsed messages. Use the functions you wrote above.

In [16]:
def read_qbbo_messages_parsed(path):
    msg_gen = read_qbbo_messages(path)

    while True:
        msg = next(msg_gen)
        yield parse_message(msg)
        

In [17]:
test_reader = read_qbbo_messages_parsed(QBBO_PATH)
assert next(test_reader) == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 3, 3, 33, 577519),
    'event': 'Start of Transmissions'
}
assert next(test_reader) == {
    'type': 'S',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 181),
    'event': 'Start of System Hours'
}
assert next(test_reader) == {
    'type': 'Q',
    'timestamp': datetime.datetime(2019, 4, 22, 4, 0, 0, 16919),
    'ticker': 'MSFT',
    'bid_price': 80.0,
    'bid_size': 100,
    'ask_price': 0.0,
    'ask_size': 0
}

In [18]:
grader.check("q5")

q5 results: All test cases passed!

Congratulations! You can now read real marketdata. Although professional trading systems use ITCH (level 2) instead of QBBO (level 1), the protocols are very similar! You can [read ITCH's spec online](https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf).

Let's do some analysis now that we can read the file.

**Question 6:** Which tickers are included in this file? Write a function that returns a set of ticker names

In [19]:
def tickers_set(path):
    out = set()
    q_encountered = False
    for message in read_qbbo_messages_parsed(path):
        if message['type'] == 'Q':
            out.add(message['ticker'])
            q_encountered = True
        elif q_encountered:
            break
    return out

In [20]:
assert len(tickers_set(QBBO_PATH)) == 4

In [21]:
grader.check("q6")

q6 results: All test cases passed!

Finally, let's plot the market of `$MSFT`. This part is ungraded:

In [ ]:
X_bid = []
Y_bid = []

X_ask = []
Y_ask = []
for message in read_qbbo_messages_parsed(QBBO_PATH):
    if 'ticker' in message and (message['timestamp'].hour < 10 or message['timestamp'].hour > 15):
        continue # Skip most out-of-market-hours messages
    if message['type'] == 'Q' and message['ticker'] == 'MSFT':
        if message['bid_price'] > 0:
            X_bid.append(message['timestamp'])
            Y_bid.append(message['bid_price'])
        if message['ask_price'] > 0:
            X_ask.append(message['timestamp'])
            Y_ask.append(message['ask_price'])
        

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(X_ask, Y_ask, label='Ask', color='blue')
plt.plot(X_bid, Y_bid, label='Bid', color='red')
plt.legend()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)